In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
cities = ['A', 'B', 'C', 'D', 'E']


In [3]:
distances = [[ 0.00, 28.02, 17.12, 27.46, 46.07],
 [28.02,  0.00, 34.00, 25.55, 25.55],
 [17.12, 34.00,  0.00, 18.03, 57.38],
 [27.46, 25.55, 18.03,  0.00, 51.11],
 [46.07, 25.55, 57.38, 51.11,  0.00]]

# Initialize Population

In [4]:
def initialize():
    population = []
    for i in range(10):
        randomSolution = cities.copy()
        random.shuffle(randomSolution)
        population.append(randomSolution)
    return np.array(population)

# Evaluation and Selection

In [5]:
def fitnessFunction(solution):
    totalDistance = 0
    for i in range(len(solution)-1):
        totalDistance += distances[cities.index(solution[i])][cities.index(solution[i+1])]

    return totalDistance

In [6]:
#Roulette Wheel Selection (others include Rank method and Tournament Size)
def evaluatePopulation(population):
    result = {}
    fitnessValues = []
    solutions = []

    for solution in population:
        fitnessValues.append(fitnessFunction(solution))
        solutions.append(solution)

    result['fitnessValues'] = fitnessValues
    centeredValues = [np.max(list(result["fitnessValues"]))-i for i in list(result["fitnessValues"])]
    result["weightedFitness"]  = [i/sum(centeredValues) for i in centeredValues]
    result["solution"] = np.array(solutions)

    return result


def RouletteWheel(population):
    
    evaluatedPopulation = evaluatePopulation(population)
    
    while (True):
        randomIndex = random.randint(0, len(population)-1)
        randomChance = evaluatedPopulation['weightedFitness'][randomIndex]

        if random.random() <= randomChance:
            pickedSolution = evaluatedPopulation['solution'][randomIndex]
            break
    
    return pickedSolution

# Combination and Mutation

In [7]:
def crossover(solution1, solution2):
    n = len(solution1)
    child = [np.nan for i in range(n)]
    num_els = np.ceil(n*(random.randint(10,90)/100))
    str_pnt = random.randint(0, n-2)
    end_pnt = n if int(str_pnt+num_els) > n else int(str_pnt+num_els)
    blockA = list(solution1[str_pnt:end_pnt])
    child[str_pnt:end_pnt] = blockA
    for i in range(n):
        if list(blockA).count(solution2[i]) == 0:
            for j in range(n):
                if pd.isna(child[j]):
                    child[j] = solution2[i]
                    break
    return child

In [8]:
def swap(solution, position1, position2):
    
    result = solution.copy()
    result[position1] = solution[position2]
    result[position2] = solution[position1] 
    return result

def swapMutation(solution):
    
    n = len(solution)
    pos_1 = random.randint(0,n-1)
    pos_2 = random.randint(0,n-1)
    
    return swap(solution, pos_1, pos_2)

# Main

In [9]:
# Create the initial population bag
population  = initialize()
# Iterate over all generations
for generation in range(200):
   # Calculate the fitness of elements in population bag
   populationFitness = evaluatePopulation(population)
   # Best individual so far
   bestFit = np.min(populationFitness["fitnessValues"])
   bestIndex = populationFitness["fitnessValues"].index(bestFit)
   bestSolution  = populationFitness["solution"][bestIndex]
   # Check if we have a new best
   if generation == 0:
      globalBestFit      = bestFit
      globalBestSolution = bestSolution
   else:
      if bestFit <= globalBestFit:
         globalBestFit      = bestFit
         globalBestSolution = bestSolution
   # Create the new population bag
   newPopulation = []
   for i in range(10):
      # Pick 2 parents from the bag
      individual1 = RouletteWheel(population)
      individual2 = RouletteWheel(population)
      new_individual = individual1
      # Crossover the parents
      if random.random() <= 0.87:
         new_element = crossover(individual1, individual2)
      # Mutate the child
      if random.random() <= 0.7:
         new_element = swapMutation(new_element) 
      # Append the child to the bag
      newPopulation.append(new_element)
   # Set the new bag as the population bag
   population = np.array(newPopulation)

In [10]:
print(globalBestSolution)

['E' 'B' 'D' 'C' 'A']
